In [183]:
import xml.etree.ElementTree as ET
import pandas as pandas
import numpy as numpy

In [184]:
import os
os.chdir("C:\\Users\\1595398\\Documents\\NAV Reporting\\Sample Reports\\XML\\")
Validations_Output = []

In [185]:
Net_Asset_Value_For_Fund = ET.parse('SDNAR022HK0287.XML')

# Extract information from Net Asset Value for Fund report. This is done by

- Creating a function called "extract_NAV_For_Fund(x) which parses the values in XML and creates 7 outputs A.Group - Group IDs 

    1. Account - Account IDs
    2. Description - Description lists of the securities
    3. Balance - Balance 
    4. Total - Total amount
    5. Net_Asset_Value - Net Asset Value as mentioned in the total summary
    6. Fund_CCY - Fund currency
    

- Call these functions and assign the output to 7 variables. These values will then be used to perform validation


In [186]:
def extract_NAV_For_Fund(x):
       
    Group = []
    for i in Net_Asset_Value_For_Fund.iter("REGR"):
        b = int(i.text)
        if b == 0:
            break
        else:
            b = b
        Group.append(b)
    
    Acc = []
    for i in Net_Asset_Value_For_Fund.iter('BNRUBR'):
        b = int(i.text)
        Acc.append(b)
    Account = [a for a in Acc if a != 0]
    
    Description = []
    for i in Net_Asset_Value_For_Fund.iter('LIB_REGR'):
        b = i.text
        Description.append(b)
        
    Bal = []
    for i in Net_Asset_Value_For_Fund.iter('MSOLDE_FM'):
        b = i.text
        if b == None:
            b = 0
        else:
            b
        Bal.append(b)
    Bal1 = [a for a in Bal if a != 0]
    Bala = []
    for r in range(0,len(Bal1)):
        x = Bal1[r].replace("(","-")
        Bala.append(x)
    Balance = []
    for i in range(0,len(Bala)):
        d = Bala[i]
        d = ''.join(e for e in d if e.isdigit() or e == '.' or e == "-")
        d = float(d)
        Balance.append(d)
    
    Tot = []
    for i in Net_Asset_Value_For_Fund.iter('CF_SOLDE_TOTAL'):
        b = i.text
        if b == None:
            b = 0
        else:
            b
        Tot.append(b)
    Tot1 = [a for a in Tot if a != 0]
    Tota = []
    for r in range(0,len(Tot1)):
        x = Tot1[r].replace("(","-")
        Tota.append(x)
    Total = []
    for i in range(0,len(Tota)):
        d = Tota[i]
        d = ''.join(e for e in d if e.isdigit() or e == '.' or e == "-")
        d = float(d)
        Total.append(d)
    
    Net_Asset_Value = Total[-1]
    
    Fund_CCY = []
    for i in Net_Asset_Value_For_Fund.iter('CMON'):
        b = i.text
        Fund_CCY.append(b)
    Fund_CCY = Fund_CCY[1]
        
    return Group, Account, Description, Balance, Total, Net_Asset_Value, Fund_CCY

In [187]:
Group, Account, Description, Balance, Total, Net_Asset_Value, Fund_CCY =  extract_NAV_For_Fund(Net_Asset_Value_For_Fund)

In [188]:
NAVFF=pandas.DataFrame([Description,Balance])

In [189]:
NAVFF=NAVFF.T

In [190]:
NAVFF.columns=['Description','Balance']

# Extract Information from Net Asset Value - Statement of Change report. This is done by
1. Extract NAV Statement of Change value 
2. Compare NAV Statement of change value with Net Asset Value

In [191]:
NAV_Statement_of_Change = ET.parse('SDNAV052HK0287.XML')

In [192]:
NAV_SoC = []
for i in NAV_Statement_of_Change.iter('CS_SOLDE_J_TOTAL'):
    b = float(i.text)
    NAV_SoC.append(b)
    
NAV_SoC_Per_Var = []
for i in NAV_Statement_of_Change.iter('CF_PC_DIF_JOURS'):
    b = float(i.text)
    NAV_SoC_Per_Var.append(b)
    
NAV_SoC_VarD = []
for i in NAV_Statement_of_Change.iter('CS_DIF_J'):
    b = float(i.text)
    NAV_SoC_VarD.append(b)    
    
NAV_SoC_Accnt = []
for i in NAV_Statement_of_Change.iter('BNRUBR'):
    b = int(i.text)
    NAV_SoC_Accnt.append(b)

In [193]:
data=pandas.DataFrame([NAV_SoC_Accnt,NAV_SoC_Per_Var])

In [194]:
data=data.T

In [195]:
data.columns=['Account','Percentage']

In [196]:
data=pandas.DataFrame(data)
for index, row in data.iterrows():
    if row[1]>.05:
        print("There is 0.5% variance in Statement of Change report, against", row['Account'], "GL")
        text = 'There is 0.5% variance in Statement of Change report against'+ str(row['Account']) + 'GL'
        Validations_Output.append(text)
    else:
        print("No variance in Statement of Change Report, against", row['Account'], "GL")
        text = 'No variance in Statement of Change Report, against' + str(row['Account']) + 'GL'
        Validations_Output.append(text)    

No variance in Statement of Change Report, against 301001.0 GL
No variance in Statement of Change Report, against 301002.0 GL
No variance in Statement of Change Report, against 302001.0 GL
No variance in Statement of Change Report, against 390000.0 GL
No variance in Statement of Change Report, against 411100.0 GL
No variance in Statement of Change Report, against 411130.0 GL
No variance in Statement of Change Report, against 421100.0 GL
No variance in Statement of Change Report, against 421130.0 GL
There is 0.5% variance in Statement of Change report, against 511100.0 GL
No variance in Statement of Change Report, against 511130.0 GL
No variance in Statement of Change Report, against 521100.0 GL
No variance in Statement of Change Report, against 521130.0 GL
No variance in Statement of Change Report, against 444444.0 GL
There is 0.5% variance in Statement of Change report, against 527100.0 GL
No variance in Statement of Change Report, against 555555.0 GL
No variance in Statement of Chang

In [197]:
if NAV_SoC[0] == Net_Asset_Value:
    print("NAV matched between NAV FOR FUND report and NAV Statement of Change report")
    text= "NAV matched between NAV FOR FUND report and NAV Statement of Change report"+  ""
    Validations_Output.append(text)
else:
    print("Amount Mismatch : NAV mis match between NAV FOR FUND report and NAV Statement of Change report")
    text = "Amount Mismatch : NAV mis match between NAV FOR FUND report and NAV Statement of Change report"+ ""
    Validations_Output.append(text)
mismatch=Net_Asset_Value-NAV_SoC[0]
print("Mismatch = ", Fund_CCY,round(mismatch,2))

NAV matched between NAV FOR FUND report and NAV Statement of Change report
Mismatch =  HKD 0.0


# Extract Information from Net Asset Value for fund. This is done by
- Extract NAV for Fund-Fund Expense Accruals
    1. INVESTMENT - EQUITIES - Total
    2. Investment - Equities - Balance
    3. Prov Unr P/L - Equities - Balance
    4. ACCRUED INCOME - Total
    5. Prov Unr FX P/L - Interest - Total
    6. CASH ASSET - Total
    7. Cash in Bank - Balance
    8. Broker-ICBC - Balance
    9. ACCRUED INCOME - Total
    10. Int Accrual - Cash Account - Balance
    11. PREPAID EXPENSE - Total
    12. Prepaid Expenses - Balance
    13. REDEMPTION PAYABLES - Total
    14. Payables on Redemption - Balance
    15. EXPENSE ACCRUALS - Total
    16. Exp Accrual - Audit Fee - Balance
    17. Exp Accrual - Custody Fee - Balance
    18. Exp Accrual - Redemption Fee - Balance
    19. Exp Accrual - Listing Expenses - Balance
    20. Exp Accrual - Publication Fee - Balance
    21. Exp Accrual - Management Fee - Balance in NAV_for_Fund_Exp_Accrls
    22. Splitting into each data point
    
- Storing NAV for Fund Accrual data in different varialbles
    1. NAV_for_Fund_Exp_Accrls as NAV_for_Fund_Exp_Accrls_Audit_Fee 
    2. NAV_for_Fund_Exp_Accrls as NAV_for_Fund_Exp_Accrls_Custody_Fee
    3. NAV_for_Fund_Exp_Accrls as NAV_for_Fund_Exp_Accrls_Redemption_Fee
    4. NAV_for_Fund_Exp_Accrls as NAV_for_Fund_Exp_Accrls_Listing_Exp
    5. NAV_for_Fund_Exp_Accrls as NAV_for_Fund_Exp_Accrls_Publication_Fee
    6. NAV_for_Fund_Exp_Accrls as NAV_for_Fund_Exp_Accrls_Management_Fee

In [198]:
NAV_for_Fund_Exp_Accrls = []
for i in Net_Asset_Value_For_Fund.iter('MSOLDE_FM'):
    b = float(i.text)
    NAV_for_Fund_Exp_Accrls.append(b)

In [199]:
NAV_for_Fund_Exp_Accrls_Audit_Fee=NAV_for_Fund_Exp_Accrls[18]
NAV_for_Fund_Exp_Accrls_Custody_Fee=NAV_for_Fund_Exp_Accrls[19]
NAV_for_Fund_Exp_Accrls_Reporting_Fee=NAV_for_Fund_Exp_Accrls[20]
NAV_for_Fund_Exp_Accrls_Transfer_Agt_Fee=NAV_for_Fund_Exp_Accrls[21]
NAV_for_Fund_Exp_Accrls_Trustee_Fee=NAV_for_Fund_Exp_Accrls[22]
NAV_for_Fund_Exp_Accrls_Management_Fee_Class_A=NAV_for_Fund_Exp_Accrls[23]
NAV_for_Fund_Exp_Accrls_Management_Fee_Class_B=NAV_for_Fund_Exp_Accrls[24]
NAV_for_Fund_Exp_Accrls_RL_Perform_Fee=NAV_for_Fund_Exp_Accrls[25]

In [200]:
import pandas as pandas

# Extract Information from Net Asset Value - Portfolio Valuation. This is done by

1.Extract following data from Portfolio Valuation 
    1. Broker-ICBC ICBC - Balance - CNY
    2. Broker-ICBC ICBC - Balance - HKD
    3. Cash in Bank STANDARD CHARTERED BANK (HONG KONG) LTD - Balance - HKD
    4. Broker-ICBC ICBC - Balance - USD
    5. Cash in Bank STANDARD CHARTERED BANK (HONG KONG) LTD - Balance - USD
    6. Exp Accrual - Management Fee - Balance
    7. Cash in Bank - Balance
    8. Exp Accrual - Audit Fee - Balance
    9. Exp Accrual - Custody Fee - Balance
    10. Exp Accrual - Listing Expenses - Balance
    11. Exp Accrual - Publication Fee - Balance
    12. Exp Accrual - Redemption Fee - Balance
    13. Int Accrual - Cash Account - Balance
    14. Payables on Redemption - Balance
    15. Prepaid Expenses - Balance
    16. Prov Unr FX P/L - Interest - Balance

In [201]:
NAV_Portfolio_Valuation = ET.parse('SDPOR072HK0287.XML')

In [202]:
NAV_Portfolio_Valuation_Total = []
for i in NAV_Portfolio_Valuation.iter('CS_TOT_COST_PTF'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Total.append(b)

NAV_Portfolio_Valuation_Total_Cost = []
for i in NAV_Portfolio_Valuation.iter('CS_TOT_MKT_PTF_SEC3'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Total_Cost.append(b)

NAV_Portfolio_Valuation_Exp_Accrls = []
for i in NAV_Portfolio_Valuation.iter('CS_TOT_COST_REF_SEC3'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Exp_Accrls.append(b)

NAV_Portfolio_Valuation_Balance  = []
for i in NAV_Portfolio_Valuation.iter('BAL_CCY_SEC3'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Balance.append(b)
    
NAV_Portfolio_Valuation_Cost_in_Fund  = []
for i in NAV_Portfolio_Valuation.iter('TOT_COST_PTF_SEC3'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Cost_in_Fund.append(b)
    
NAV_Portfolio_Valuation_Market_Value_in_Fund  = []
for i in NAV_Portfolio_Valuation.iter('MKT_VALUE_PTF_SEC3'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Market_Value_in_Fund.append(b)

NAV_Portfolio_Valuation_Value_Ref  = []
for i in NAV_Portfolio_Valuation.iter('VALUE_REF_SEC3'):
    b = i.text
    NAV_Portfolio_Valuation_Value_Ref.append(b)
    
NAV_Portfolio_Valuation_Accrued_Interest = []
for i in NAV_Portfolio_Valuation.iter('INT_CCY_SEC3'):
    b = i.text
    NAV_Portfolio_Valuation_Accrued_Interest.append(b)

NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy = []
for i in NAV_Portfolio_Valuation.iter('CS_TOT_MKT_PTF_SEC3'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy.append(b)
    
NAV_Portfolio_Valuation_Total_Investment = []
for i in NAV_Portfolio_Valuation.iter('CS_TOT_COST_PTF_REP'):
    b = float(i.text)
    NAV_Portfolio_Valuation_Total_Investment.append(b)
    
NAV_Portfolio_Valuation_Description = []
for i in NAV_Portfolio_Valuation.iter('LIBELLE_SEC3'):
    b = i.text
    NAV_Portfolio_Valuation_Description.append(b)

In [203]:
NAV_Portfolio_Valuation_Markt_Value_Management_Fee_Class_A=NAV_Portfolio_Valuation_Market_Value_in_Fund[3]
NAV_Portfolio_Valuation_Markt_Value_Management_Fee_Class_B=NAV_Portfolio_Valuation_Market_Value_in_Fund[4]
NAV_Portfolio_Valuation_Markt_Value_RL_Perform_Fee=NAV_Portfolio_Valuation_Market_Value_in_Fund[5]
NAV_Portfolio_Valuation_Markt_Value_Acc_Audit_Fee=NAV_Portfolio_Valuation_Market_Value_in_Fund[6]
NAV_Portfolio_Valuation_Markt_Value_Acc_Reporting_Fee=NAV_Portfolio_Valuation_Market_Value_in_Fund[7]
NAV_Portfolio_Valuation_Markt_Value_Acc_transfer_Agt_Fee=NAV_Portfolio_Valuation_Market_Value_in_Fund[8]
NAV_Portfolio_Valuation_Markt_Value_Acc_trustee_Fee=NAV_Portfolio_Valuation_Market_Value_in_Fund[9]
NAV_Portfolio_Valuation_Markt_Value_Acc_Custody_Fee=NAV_Portfolio_Valuation_Market_Value_in_Fund[10]
NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Cash_at_Bank=NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy[0]
NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Pre_Exp=NAV_Portfolio_Valuation_Cost_in_Fund[17]
NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Receivalbles=NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy[5]
NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Payables=NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy[4]

In [204]:
NAVPV=pandas.DataFrame([NAV_Portfolio_Valuation_Description,NAV_Portfolio_Valuation_Balance])

In [205]:
NAVPV=NAVPV.T

In [206]:
NAVPV.columns=['Description','Balance']

# Validation between NAV for Fund with Portfolio Valuation

In [207]:
NAVFF=pandas.DataFrame(NAVFF)
NAVPV=pandas.DataFrame(NAVPV)
for index, i in NAVFF.iterrows():
    for index, j in NAVPV.iterrows():
        if str.lstrip(str.rstrip(i['Description']))==str.lstrip(str.rstrip(j['Description'])) and i['Balance']==j['Balance']:
            print(i['Description'], "matched between Porfolio Valuation report and Net asset value for fund for the amount", i['Balance'])
            text = i['Description']+ "matched between Porfolio Valuation report and Net asset value for fund for the amount"+ str(i['Balance'])
            Validations_Output.append(text)
print(i['Description'],'-',"Not Match between Porfolio Valuation report and Net asset value for fund ")
text = i['Description'] + '-' + "Not Match between Porfolio Valuation report and Net asset value for fund" + "" 
Validations_Output.append(text)

Prepaid Exps - Preliminary Exp matched between Porfolio Valuation report and Net asset value for fund for the amount 321112.01
Dividend Receivable  matched between Porfolio Valuation report and Net asset value for fund for the amount 448157.62
Other Payables  matched between Porfolio Valuation report and Net asset value for fund for the amount -10.0
Exp Accrual - Audit Fee  matched between Porfolio Valuation report and Net asset value for fund for the amount -205578.15
Exp Accrual - Reporting Fee  matched between Porfolio Valuation report and Net asset value for fund for the amount -74138.61
Exp Accrual - Transfer Agt Fee matched between Porfolio Valuation report and Net asset value for fund for the amount -84.78
Exp Accrual - Trustee Fee  matched between Porfolio Valuation report and Net asset value for fund for the amount -60083.07
Exp Accrual - Mgmt Fee Class A matched between Porfolio Valuation report and Net asset value for fund for the amount -126374.67
Exp Accrual - Mgmt Fee Cla

In [208]:
CIB = 0
#NAVFF=pandas.DataFrame(NAVFF)
NAVPV=pandas.DataFrame(NAVPV)
#for index, i in NAVFF.iterrows():
for index, j in NAVPV.iterrows():
    if j['Description']=="Cash in Bank STANDARD CHARTERED BANK (HONG KONG) LTD. ":
        CIB=j['Balance'] + CIB
        print(j['Balance'])

30.39
9572020.9
1.16


In [209]:
NAVFF=pandas.DataFrame(NAVFF)
NAVPV=pandas.DataFrame(NAVPV)
for index, i in NAVFF.iterrows():
    if str.lstrip(str.rstrip(i['Description']))=="Cash in Bank" and i['Balance']==CIB:
        print(i['Description'],"Matched between Portfolio valuation report and Net asset value for fund")
        text = i['Description'],"Matched between Portfolio valuation report and Net asset value for fund" + "" 
        Validations_Output.append(text)
print("Cash in Bank not matched between Porfolio Valuation report and Net asset value for fund")
text = "Cash in Bank not matched between Porfolio Valuation report and Net asset value for fund" + "" 
Validations_Output.append(text)

Cash in Bank not matched between Porfolio Valuation report and Net asset value for fund


# Extraction of General Ledger Trial Balance

In [210]:
NAV_GL_TB = ET.parse('SHJNA102HK0287.XML')

In [211]:
GL_TB_Acc = []
for i in NAV_GL_TB.iter('BNRUBR'):
    b = int(i.text)
    GL_TB_Acc.append(b)
    
GL_TB_Cry = []
for i in NAV_GL_TB.iter('ACMON'):
    b = (i.text)
    GL_TB_Cry.append(b)

GL_TB_Acc_Bal = []
for i in NAV_GL_TB.iter('CF_ABAL'):
    b = float(i.text)
    GL_TB_Acc_Bal.append(b)

GL_TB_Acc_Fund = []
for i in NAV_GL_TB.iter('CF_CBAL'):
    b = float(i.text)
    GL_TB_Acc_Fund.append(b)

GL_TB_Acc_Ttl_Asset_Liab = []
for i in NAV_GL_TB.iter('CP_TOTAL_SDNAR02'):
    b = float(i.text)
    GL_TB_Acc_Ttl_Asset_Liab.append(b)

GL_TB_Acc_Ttl_Cap_PL = []
for i in NAV_GL_TB.iter('CP_TOTAL_SDNAV05'):
    b = float(i.text)
    GL_TB_Acc_Ttl_Cap_PL.append(b)

GL_TB_Acc_Closing_Balance = []
for i in NAV_GL_TB.iter('CF_TOTALBALANCE'):
    b = float(i.text)
    GL_TB_Acc_Closing_Balance.append(b)
    
    
GL_TB_Acc_Total_Assests_Liab = []
for i in NAV_GL_TB.iter('CP_TOTAL_SDNAR02'):
    b = float(i.text)
    GL_TB_Acc_Total_Assests_Liab.append(b)

# Validation of General Ledger Trial Balance

In [212]:
D = (pandas.DataFrame(GL_TB_Acc))
D["Currency"] = pandas.DataFrame(GL_TB_Cry)

In [213]:
D.head()

,0,Currency
0,101100,HKD
1,101130,HKD
2,120121,HKD
3,132100,HKD
4,132150,HKD


In [214]:
Array = D.loc[(D[0]>400000) & (D[0]<600000)]

In [215]:
if (((Array.loc[(Array["Currency"]!=Fund_CCY)]).count())[0] == 0):
    print("No Exceptions found")
    text="No Exceptions Found between GL Trial Balance and Fund Currency" + "" 
    Validations_Output.append(text)
else:
    print("Exceptions found")
    text="Exceptions Found between GL Trial Balance and Fund Currency" + "" 
    Validations_Output.append(text)

No Exceptions found


In [216]:
if GL_TB_Acc_Closing_Balance[0] < 1:
    print("No Exceptions found - Closing Balance is zero")
    text="No Exceptions Found between GL Trial Balance Closing Balance is Zero" +  "" 
    Validations_Output.append(text)
else:
    print("Exceptions found")
    text = "Exceptions Found between GL Trial Balance Closing Balance" + "" 
    Validations_Output.append(text)

No Exceptions found - Closing Balance is zero


In [217]:
Net_Asset_Value

264393156.59

In [218]:
Check=GL_TB_Acc_Total_Assests_Liab[0] - Net_Asset_Value
if Check < 1:
    print("No Exceptions found - NAV is equal to Total Assets & Liabilities")
    text = "No Exceptions Found NAV is equal to Total Assets & Liabilities", " + "" " 
    Validations_Output.append(text)
else: 
    print("Exceptions found - NAV mismatch with Total Assets & Liabilities")
    text = "Exceptions Found - NAV mismatch with Total Assets & Liabilities" + "" 
    Validations_Output.append(text)

No Exceptions found - NAV is equal to Total Assets & Liabilities


In [219]:
Array

,0,Currency
26,411100,HKD
27,411130,HKD
28,421100,HKD
29,421130,HKD
30,431100,HKD
31,431130,HKD
32,441100,HKD
33,441130,HKD
34,444444,HKD
35,451100,HKD


# Extraction and Validation of Dividend Receivable Report

In [220]:
NAV_DRR = ET.parse('SDCOU042HK0287.XML')

In [221]:
NAV_DRR_Security_Name = []
for i in NAV_DRR.iter('NOMVAL'):
    b = i.text
    NAV_DRR_Security_Name.append(b)

In [222]:
Leng = len(NAV_DRR_Security_Name)
for i in range(0,Leng):
    if i == None:
        print("No Exception Found in Dividend Receivable Report")
        text = "No Exceptions Found in Dividend Receivable Report" + "" 
        Validations_Output.append(text)
    else:
        print("Check No of shares are correct with Bloomberg report")
        text = "Check No of shares are correct with Bloomberg report" + "" 
        Validations_Output.append(text)

Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report
Check No of shares are correct with Bloomberg report


# Extraction and Validation of Realised Gain and Loss Report

In [223]:
NAV_VRGLR = ET.parse('SDPOR092HK0287.XML')

In [224]:
NAV_VRGLR_FXGL = []
for i in NAV_VRGLR.iter('SUMGAIN_PERTE_TCHGPERREPORT'):
    b = float(i.text)
    NAV_VRGLR_FXGL.append(b)

In [225]:
if NAV_VRGLR_FXGL[0]==NAV_SoC_VarD[2]:
    print("No Expections between Realised gain and loss report")
    text = "No Expections between Realised gain and loss report" + "" 
    Validations_Output.append(text)
else:
    print("There is a difference in Realized FX loss")
    text = "There is a difference in Realized FX loss" + "" 
    Validations_Output.append(text)

No Expections between Realised gain and loss report


# Extraction from Balance Sheet & Profit &  Loss Account

This is done by
Extract following data from Balance Sheet & Profit & Loss Account 
    1. Assets
    2. Liabilities

In [226]:
BSheet_Profit_Loss = ET.parse('SHJNA112HK0287.XML')

In [227]:
BSheet_Profit_Loss_Accnt_Assets  = []
for i in BSheet_Profit_Loss.iter('CF_DEBIT_BAL'):
    d1 = i.text
    if d1 == None:
        d1 = 0
        b = (d1)
        
    else:
        b = float(d1)
    BSheet_Profit_Loss_Accnt_Assets.append(b)
    
BSheet_Profit_Loss_Accnt_Liabilities  = []
for i in BSheet_Profit_Loss.iter('CF_CREDIT_BAL'):
    d1 = i.text
    if d1 == None:
        d1 = 0
        b = (d1)
        
    else:
        b = float(d1)
    BSheet_Profit_Loss_Accnt_Liabilities.append(b)

In [228]:
BSheet_Profit_Loss_Accnt_Assets_Cost_Equities=BSheet_Profit_Loss_Accnt_Assets[0]
BSheet_Profit_Loss_Accnt_Assets_Prepaid_Expenses=BSheet_Profit_Loss_Accnt_Assets[1]
BSheet_Profit_Loss_Accnt_Assets_Receivables=BSheet_Profit_Loss_Accnt_Assets[2]
BSheet_Profit_Loss_Accnt_Assets_Cash_at_Bank=BSheet_Profit_Loss_Accnt_Assets[3]
BSheet_Profit_Loss_Accnt_Liabilities_Payables=BSheet_Profit_Loss_Accnt_Liabilities[5]

# Validation of Balance Sheet & Profit &  Loss Account

In [229]:
if NAV_Portfolio_Valuation_Total_Investment[0] == BSheet_Profit_Loss_Accnt_Assets_Cost_Equities:
    print("Cost equities matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Cost equities matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)
else:
    print("Cost equities mis match between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Cost equities mis match between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)

Cost equities matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account


In [230]:
if NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Cash_at_Bank == BSheet_Profit_Loss_Accnt_Assets_Cash_at_Bank:
    print("Cash at Bank matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Cash at Bank matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)
else:
    print("Cash at Bank mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Cash at Bank mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)

Cash at Bank matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account


In [231]:
if NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Pre_Exp == BSheet_Profit_Loss_Accnt_Assets_Prepaid_Expenses:
    print("Prepaid Expenses matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Prepaid Expenses matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)
else:
    print("Prepaid Expenses mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Prepaid Expenses mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)

Prepaid Expenses matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account


In [232]:
if NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Receivalbles == BSheet_Profit_Loss_Accnt_Assets_Receivables:
    print("Receivables matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Receivables matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)
else:
    print("Receivables mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Receivables mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)

Receivables matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account


In [233]:
if NAV_Portfolio_Valuation_Total_Cost_Fund_Ccy_Payables == BSheet_Profit_Loss_Accnt_Liabilities_Payables:
    print("Payables matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Payables matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)
else:
    print("Payables mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account")
    text = "Payables mismatch between NAV Portfolio Valuation and BSheet and Profit and Loss Account" + "" 
    Validations_Output.append(text)

Payables matched between NAV Portfolio Valuation and BSheet and Profit and Loss Account


In [234]:
Validations_Output = pandas.DataFrame(Validations_Output)

In [235]:
Validations_Output = Validations_Output.rename(columns = {0 : "Results"})

In [236]:
Validations_Output.to_csv("Validations_Output.csv", index = True)

In [237]:
Validations_Output

,Results
0,"No variance in Statement of Change Report, aga..."
1,"No variance in Statement of Change Report, aga..."
2,"No variance in Statement of Change Report, aga..."
3,"No variance in Statement of Change Report, aga..."
4,"No variance in Statement of Change Report, aga..."
5,"No variance in Statement of Change Report, aga..."
6,"No variance in Statement of Change Report, aga..."
7,"No variance in Statement of Change Report, aga..."
8,There is 0.5% variance in Statement of Change ...
9,"No variance in Statement of Change Report, aga..."
